# VAR-Model for money supply M2, inflation rate and gdp in the US

### Importing the relevant libraries

In [263]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

### Importing the relevant datasets

In [264]:
money = pd.read_csv("M2SL.csv") # money supply M2 in billion US-dollars

gdp = pd.read_csv("GDP.csv") # US GDP per capita

inflation = pd.read_csv("inflation.csv") # Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (1982-1984 = 100)

### Cleaning the datasets

In [265]:
# Changing columns names

money = money.rename(columns={"DATE": "date", "M2SL": "M2"}).copy()

inflation = inflation.rename(columns={"DATE": "date", "CPIAUCSL": "inflation"}).copy()

gdp = gdp.rename(columns={"DATE": "date", "GDP": "gdp"}).copy()

In [266]:
# Changing the date-column to datetime-format

money["date"] = pd.to_datetime(money["date"]).copy()

gdp["date"] = pd.to_datetime(gdp["date"]).copy()

inflation["date"] = pd.to_datetime(inflation["date"]).copy()

### Setting all datasets to the same frequency and time-span

In [267]:
money_min = money["date"].min()
money_max = money["date"].max()

gdp_min = gdp["date"].min()
gdp_max = gdp["date"].max()

inflation_min = inflation["date"].min()
inflation_max = inflation["date"].max()

In [268]:
df_min = pd.DataFrame({"min": [money_min, gdp_min, inflation_min]},
                      index = ["money_min", "gdp_min", "inflation_min"])

df_max = pd.DataFrame({"max": [money_max, gdp_max, inflation_max]},
                      index = ["money_max", "gdp_max", "inflation_max"])

In [269]:
df_min["min"].max() # 1960-01-01 will be our threshold on the bottom

Timestamp('1959-01-01 00:00:00')

In [270]:
df_max["max"].min() # 2023-01-01 will be our threshold on top

Timestamp('2024-04-01 00:00:00')

In [271]:
# Deleting all rows above or below our thresholds

money = money.iloc[:784].copy()

inflation = inflation[144:928].copy()
inflation.reset_index(inplace=True)

gdp = gdp.iloc[48:].copy()
gdp.reset_index(inplace=True)

In [272]:
inflation.drop(columns=["index"], inplace=True)
gdp.drop(columns=["index"], inplace=True)

In [273]:
# Deleting the rows, so that every time-series has the same frequency

money = money.iloc[::3].copy()
inflation = inflation[::3].copy()

In [274]:
# Setting the date as index-column

money = money.set_index("date").copy()

gdp = gdp.set_index("date").copy()

inflation = inflation.set_index("date").copy()